In [ ]:

import os
import pandas as pd

#Load the .csv file
csv_file = 'C:\\Users\\Ashle\\OneDrive\\Desktop\\Lab codes\\2025 codes\\carbenoxolone data'

#Read CSV file
data = pd.read_csv(csv_file) #reads the file
data() #displays the data